In [2]:
import pandas as pd
import numpy as np
import time
import os

import spacy

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
data_folder = "../Data-Analytics-DRMIS-SAP-Production-DATA/"

df = pd.read_csv(data_folder + 'vehicle_df_interest.csv', index_col = 0).reset_index(drop = True)

In [66]:
df[50:100]

,Order,Description,Total planned costs,PM Orders planned hours,Technical Status,PM Orders actual hours,Tail Number / Identifier,User Status,Equipment,MaintActivityType,Main work center,Vehicle Type,Utilizer,Actual release date,Basic finish date,days WIP
50,1101847731,Repairs After Inspection,0,0.0,SERV,44.0,2005-64558,OAPP,20110431,L01,AG-LS-V2,EV0B22,102,27/11/2015,28/01/2016,62
51,1102572807,*64558 PTO NOT ENGAGING/ BARCKET BROKE,560.6,5.0,SERV,6.0,2005-64558,OAPP,20110431,L01,AG-LS-V2,EV0B22,102,27/02/2020,12/06/2020,106
52,1102311423,64558 - Repair Tarp system,750.26,7.0,SERV,7.0,2005-64558,OAPP,20110431,L01,AG-O--M2,EV0B22,102,24/07/2018,24/07/2018,0
53,1102187923,Repl Air Dryer,0,0.0,SERV,5.0,2005-64559,OAPP,20110387,L01,AG-LS-V2,EV0B22,102,26/10/2017,26/10/2017,0
54,1102500653,TRUCK DUMP S/C INSP-12 MONTH,493.43,4.5,SERV,4.0,2005-64559,OAPP,20110387,L27,AG-LS-V2,EV0B22,102,04/10/2019,09/01/2020,97
55,1102506268,64559 Repairs after inspection,"1,480.28",13.5,SERV,10.0,2005-64559,OAPP,20110387,L01,AG-LS-V2,EV0B22,102,08/10/2019,14/02/2020,129
56,1102349068,64559 repairs after,964.62,9.0,SERV,5.0,2005-64559,OAPP,20110387,L01,AG-LS-V2,EV0B22,102,17/10/2018,26/11/2018,40
57,1101995776,TRUCK DUMP S/C INSP-12 MONTH,418.28,4.0,SERV,2.5,2005-64559,OAPP,20110387,L27,AG-LS-V2,EV0B22,102,03/10/2016,03/11/2016,31
58,1102227143,Repairs After Inspection,"12,689.51",121.0,SERV,64.5,2005-64559,OAPP,20110387,L01,AG-LS-V2,EV0B22,102,26/01/2018,09/04/2018,73
59,1101925611,TEME SPV- broken Air supply valve,0,0.0,SERV,1.5,2005-64559,OAPP,20110387,L01,AG-LS-V2,EV0B22,102,28/04/2016,28/04/2016,0


In [7]:
text = df['Description'].tolist()
token = Tokenizer()
token.fit_on_texts(text)
vocab = len(token.index_word) + 1
encoded_text = token.texts_to_sequences(text)

In [64]:
max_length = max([len(x) for x in encoded_text])
max_length

11

In [23]:
token_dict = token.__dict__

In [24]:
token_dict.keys()

dict_keys(['word_counts', 'word_docs', 'filters', 'split', 'lower', 'num_words', 'document_count', 'char_level', 'oov_token', 'index_docs', 'word_index', 'index_word'])

In [61]:
word_list1 = list(token_dict['index_word'].values())
drop_list = []

remove:
- non words (codes, ect)
- numbers
- stop words

In [16]:
print(help(token))

Help on Tokenizer in module keras_preprocessing.text object:

class Tokenizer(builtins.object)
 |  Text tokenization utility class.
 |  
 |  This class allows to vectorize a text corpus, by turning each
 |  text into either a sequence of integers (each integer being the index
 |  of a token in a dictionary) or into a vector where the coefficient
 |  for each token could be binary, based on word count, based on tf-idf...
 |  
 |  # Arguments
 |      num_words: the maximum number of words to keep, based
 |          on word frequency. Only the most common `num_words-1` words will
 |          be kept.
 |      filters: a string where each element is a character that will be
 |          filtered from the texts. The default is all punctuation, plus
 |          tabs and line breaks, minus the `'` character.
 |      lower: boolean. Whether to convert the texts to lowercase.
 |      split: str. Separator for word splitting.
 |      char_level: if True, every character will be treated as a token.

In [ ]:
#create ranges of days WIP